In [ ]:
import tiktoken
import regex as re
import pandas as pd


def encode_text(text):
    enc = tiktoken.encoding_for_model("gpt-4")
    tokenized_text = enc.encode(text)
    return tokenized_text


def decode_text(text):
    enc = tiktoken.encoding_for_model("gpt-4")
    decoded_tokens = [enc.decode([token]) for token in text]
    return decoded_tokens


def apply_token(text):
    enc = tiktoken.encoding_for_model("gpt-4")
    tokenized_text = [enc.decode([token]) for token in enc.encode(text)]
    return tokenized_text



def concatenate_sentences(answer):
    new_answer = []
    index = 0
    while index < len(answer) - 1:
        sentence = answer[index]
        if re.search(r':\n\n\d\.', sentence):
            if len(sentence) <= 90:
                new_answer.append(sentence + answer[index + 1])
                index += 1
            else:
                new_answer.append(sentence)
        else:
            new_answer.append(sentence)
        index += 1
    return new_answer


def heuristic(answer):
    sentences = []
    decoded = decode_text(encode_text(answer))
    lengths = [len(dec) for dec in decoded]
    index = 0
    tokens_in_streak = 0
    while index < len(lengths):
        if tokens_in_streak >= 10 and lengths[index] == 1:
            if lengths[index-1] == 3:
                sentences.append("".join(decoded[:tokens_in_streak]))
                decoded = decoded[tokens_in_streak:]
            elif lengths[index-1] == 1:
                sentences.append("".join(decoded[:tokens_in_streak-1]))
                decoded = decoded[tokens_in_streak-1:]
                index -= 1
            else:
                sentences.append("".join(decoded[:tokens_in_streak+1]))
                decoded = decoded[tokens_in_streak+1:]
                index += 1
            tokens_in_streak = 0
        else:
            index += 1
            tokens_in_streak += 1
    else:
        if tokens_in_streak > 0:
            sentences.append("".join(decoded))
    answer = concatenate_sentences(answer)
    return sentences

In [ ]:

def take_first(answer):
    if type(answer) == str:
        return heuristic(answer)[0]


old = pd.read_csv("subset_test_pairs.csv")
old["Response"] = old["Response"].apply(take_first)
old.to_csv("subset_test_pairs.csv", index=False)
